In [1]:
# Import modules
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)
import tensorflow as tf

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./t10k-labels-idx1-ubyte.gz


In [2]:
# Parameters
learning_rate = 0.001
training_epochs = 20
batch_size = 64
display_step = 1

n_input = 784
n_classes = 10
n_hidden_layer = 256

In [9]:
# Store layers weight and bias
weights = {
    'hidden_layer': tf.Variable(tf.random_normal([n_input, n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_hidden_layer, n_classes]))
}
biases = {
    'hidden_layer': tf.Variable(tf.random_normal([n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [10]:
# tf Graph input
x = tf.placeholder('float', [None, 28, 28, 1])
y = tf.placeholder('float', [None, n_classes])

x_flat = tf.reshape(x, [-1, n_input])

In [11]:
# Hidden layer with Relu activation
layer_1 = tf.add(tf.matmul(x_flat, weights['hidden_layer']), biases['hidden_layer'])
layer_1 = tf.nn.relu(layer_1)
# Output layer with linear activation
logits = tf.add(tf.matmul(layer_1, weights['out']), biases['out'])

In [24]:
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Calculate accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [15]:
# Init the variables
init = tf.global_variables_initializer()

In [16]:
save_file = './train_model.ckpt'
saver = tf.train.Saver()

In [18]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for e in range(training_epochs):
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op and cost op
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        # Display logs per epoch step:
        if e % display_step == 0:
            c = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print("Epoch:", '%04d' % (e+1), "cost=", \
                "{:.9f}".format(c))
    saver.save(sess, save_file)
    print('Optimization Finished')

Epoch: 0001 cost= 27.199958801
Epoch: 0002 cost= 10.915661812
Epoch: 0003 cost= 9.740331650
Epoch: 0004 cost= 9.563018799
Epoch: 0005 cost= 9.244540215
Epoch: 0006 cost= 10.153213501
Epoch: 0007 cost= 6.482011318
Epoch: 0008 cost= 6.544493675
Epoch: 0009 cost= 4.106618881
Epoch: 0010 cost= 5.669189453
Epoch: 0011 cost= 5.049091339
Epoch: 0012 cost= 4.519397736
Epoch: 0013 cost= 7.805425644
Epoch: 0014 cost= 6.076913357
Epoch: 0015 cost= 3.935233355
Epoch: 0016 cost= 2.370515347
Epoch: 0017 cost= 4.745139122
Epoch: 0018 cost= 3.696955919
Epoch: 0019 cost= 6.509666443
Epoch: 0020 cost= 1.821187735
Optimization Finished


Test model

In [26]:
# Restpre saved model above
saver = tf.train.Saver()

# Launch the graph
with tf.Session() as sess:
    saver.restore(sess, save_file)

    test_accuracy = sess.run(
        accuracy,
        feed_dict={x: mnist.test.images, y: mnist.test.labels})

print('Test Accuracy: {}'.format(test_accuracy))

INFO:tensorflow:Restoring parameters from ./train_model.ckpt
Test Accuracy: 0.8661999702453613
